# Spark 数据降维
不同于回归、分类、聚类，降维方法并不是用来做模型预测的，它本身是一种预处理方法，或则说是一种特征转换的方法，而不是模型预测的方法。
## 降维方法的种类
MLlib提供两种相似的降维模型：PCA（principal components Analysis,主成分分析）和SVD（singular Value Decomposition,奇异值分解法）  
### 1 主成分分析
PCA处理一个数据矩阵，抽取矩阵中K个主向量，主向量彼此不相关。计算结果中，第一个主向量表示输入数据的最大变化方向。之后的每个主向量依次代表不考虑之前计算过的所有方向时最大的变化方向。  
因此，返回的K个主成分代表了输入数据可能的最大变化。事实上，每一个主成分向量上有着和原始数据矩阵相同的维度。因此需要使用映射来做一次降维，原来的数据被投影到主向量表示的K维空间。
### 2 奇异值分解
SVD试图将一个m*m的矩阵分解为三个主成分矩阵：  
X ＝ U*S*Vt  
一般情况下只保留K个奇异值，他们能代表数据的最主要变化，剩余的奇异值被丢弃。、
## 本章节主要分为以下几个部分
* 一、从数据中抽取合适的特征
* 二、训练降维模型
* 三、使用降维模型
* 四、评级降维模型

## 一、从数据中抽取合适的特征
在我们到目前为止所学到所有机器学习模型中，降维模型还可以产生数据的特征向量表示。  
本章我们将利用户外脸部标注集（LFW，Labeled Faces in the Wild）深入到图像处理的世界。这个数据集包含13000多张主要从互联网上获得的公众人物的面部图片。这些图片用人名进行了标注。  
### 从LFW数据集中提取特征
为了避免下载处理非常大的数据，我们只处理图片集的一个子集，选择以A字母开头的人的面部图片。通过下面的链接可以下载到这个数据集：http://vis-www.cs.umass.edu/lfw/lfw-a.tgz

In [1]:
val PATH = "file:///Users/lzz/work/SparkML/"
val path = PATH + "data/lfw/*"

In [2]:
val rdd = sc.wholeTextFiles( path )
val first = rdd.first
println( first )

(file:/Users/lzz/work/SparkML/data/lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg,���� JFIF      �� C 


��  � �" ��           	
�� �   } !1AQa"q2���#B��R��$3br�	
%&'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz���������������������������������������������������������������������������        	
�� �  w !1AQaq"2�B����	#3R�br�
nr��Ii��
���� ��g7ȮTUغ�n.K"�rEt�"ب� OU��R��"�$TvzlC/FC�����>U9��SKR��������J{kd��F0UT�V�9�!裥Y�� ���Ysثy�����QYs�[ B���We� ��,���F��wM��f+o����jA+X�MT���Gҹ��NK�
�m>��^���X�j�JƖ�rv�Ê鄎ۡ����8�^�]����&�6��L���+^c#N(�����ҳﴹc$��C4�N1�5�e3G�r"�H9NBT��W85�_Y��\`��D��=+t�&�F9�K�F	�1F�TH�A��O�J�@�1�K����^�@)r*V����_�.`xJ������z�` g4��w�`�0�5HQE�)ri(c����ZO�tn+��i��G�t��O��:���v��s^��r!|F�c+���M�����J��U�Zid<�	�#8�6���ː�Vc<�������]"���c�L��T�/�z

我们的例子将使用自定义代码来读取图片，所以我们需要文件路径这部分。因此我们通过下面的map函数删除前面部分：

In [3]:
val files = rdd.map { case (fileName, content) => fileName.replace( "file:", "") }
println( files.first )

/Users/lzz/work/SparkML/data/lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg


In [4]:
println( files.count )

1054


### 提取面部图片作为向量
***（1）***载入图片  
第一个函数是从文件中读取图片：

In [5]:
import java.awt.image.BufferedImage
def loadImageFromFile( path: String ): BufferedImage = {
    import javax.imageio.ImageIO
    import java.io.File
    ImageIO.read( new File( path ))
}

这将返回一个java.awt.image.BufferedImage类的实例，存储图片数据并提供很多有用的方法。

In [6]:
val aePath = "/Users/lzz/work/SparkML/data/lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg"
println(aePath)
val aeImage = loadImageFromFile( aePath )
aeImage

/Users/lzz/work/SparkML/data/lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg


2

***（2）*** 转换灰度图片并改变图片尺寸  
我们定义自己的处理函数。我们将使用java.awt.image包一步做完灰度转换和尺寸改变：

In [7]:
def processImage( image: BufferedImage, width: Int, height: Int): BufferedImage = {
    val bwImage = new BufferedImage( width, height, BufferedImage.TYPE_BYTE_GRAY )
    val g = bwImage.getGraphics()
    g.drawImage( image, 0, 0, width, height, null)
    g.dispose()
    bwImage
}

In [8]:
val grayImage = processImage( aeImage, 100, 100)
grayImage

0

In [9]:
import javax.imageio.ImageIO
import java.io.File
ImageIO.write( grayImage, "jpg", new File("/tmp/aeGray.jpg"))

true

***（3）***提取特征向量  
处理流程的最后一步是提取真实的特征向量作为我们降维模型的输入。正如之前提到的，纯灰度像素数据将作为特征。我们将通过打平二维的像素矩阵来构造一维的向量。BufferedImage类为此提供了一个工具方法，可以在我们的函数中使用：

In [10]:
def getPixelsFromImage( image: BufferedImage): Array[Double] = {
    val width = image.getWidth
    val height = image.getHeight
    val pixels = Array.ofDim[Double]( width * height )
    image.getData.getPixels(0, 0, width, height, pixels)
}

之后我们中一个功能函数中组合这三个函数，接受一个图片文件位置和需要处理的宽和高，返回一个包含像素数据的Array[Double]值：

In [11]:
def extractPixels( path: String, width: Int, height: Int): Array[Double] = {
    val raw = loadImageFromFile(path)
    val processed = processImage( raw, width, height )
    getPixelsFromImage( processed )
}

把这个函数应用到包含图片路径的RDD的每一个元素上将产生一个新的RDD，新的RDD包含每张图片等像素数据。让我们通过下面的代码看看开始的几个元素：

In [12]:
val pixels = files.map( f => extractPixels(f, 50, 50))
println( pixels.take(10).map(_.take(10).mkString("", ",", ", ...")).mkString("\n"))

1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0, ...
247.0,173.0,159.0,144.0,139.0,155.0,32.0,7.0,4.0,5.0, ...
253.0,254.0,253.0,253.0,253.0,253.0,253.0,253.0,253.0,253.0, ...
242.0,242.0,246.0,239.0,238.0,239.0,225.0,165.0,140.0,167.0, ...
47.0,221.0,205.0,46.0,41.0,154.0,127.0,214.0,232.0,232.0, ...
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0, ...
75.0,76.0,72.0,72.0,72.0,74.0,71.0,78.0,54.0,26.0, ...
25.0,27.0,24.0,22.0,26.0,27.0,19.0,16.0,22.0,25.0, ...
240.0,240.0,240.0,240.0,240.0,240.0,240.0,240.0,240.0,240.0, ...
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0, ...


最后一步是为每一张图片创建MLlib向量对象。我们将缓存RDD来加速我们之后的计算：

In [13]:
import org.apache.spark.mllib.linalg.Vectors
val vectors = pixels.map( p => Vectors.dense(p) )
vectors.setName( "image-vectors" )
vectors.cache

image-vectors MapPartitionsRDD[4] at map at <console>:36

### 4、正则化
在运行降维模型尤其是PCA之前，通常会对输入数据进行标准化。

In [14]:
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.feature.StandardScaler
val scaler = new StandardScaler( withMean = true, withStd = false).fit(vectors)

In [15]:
val scaledVectors = vectors.map( v => scaler.transform(v) )

## 二、训练降维模型
MLlib中的降维模型需要向量作为输入。但是，并不像聚类直接处理RDD[Vector],PCA和SVD的计算是通过提供基于RowMatrix的方法实现的（区别主要是语法的不同，RowMatrix也仅仅是一个RDD[Vector]的简单封装）。
###  在LFW数据集上运行PCA
因为我们已经从图像的像素数据中提取出了向量，现在可以初始化一个新的RowMatrix, 并调用computePrincipalComponents来计算我们的分布式矩阵的前K个主成分：

In [16]:
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.distributed.RowMatrix
val matrix = new RowMatrix( scaledVectors )
val K = 10
val pc = matrix.computePrincipalComponents(K)

#### 1、可视化特征脸
现在我们已经训练了自己的PCA模型，但结果如何？让我们分析一下结果矩阵的不同维度：

In [17]:
val rows = pc.numRows
val cols = pc.numCols
println( rows, cols)

(2500,10)


In [19]:
import breeze.linalg.DenseMatrix
val pcBreeze = new DenseMatrix( rows, cols, pc.toArray )

Breeze的linalg包中提供了使用的函数吧矩阵写入CSV文件中。我们将使用它把主成分保存为临时CSV文件：

In [21]:
import breeze.linalg.csvwrite
csvwrite( new File("/tmp/pc.csv"), pcBreeze)

##  三、使用降维模型
### 在LFW数据集上使用PCA投影数据
我们将通过把每一个LFW图像投影到10维的向量上来演示这个概念。用矩阵乘法把图像矩阵和主成分矩阵相乘来实现投影。因为图像矩阵是分布式的MLlib RowMatrix。Spark帮助我们实现了分布式计算的multipy函数：

In [22]:
val projected = matrix.multiply(pc)
println( projected.numRows, projected.numCols )

(1054,10)


In [23]:
println( projected.rows.take(5).mkString("\n") )

[2656.255334446328,1331.4316152623849,443.77171439925996,-352.5378024086936,52.35190158301246,377.3800577741128,487.0249575522312,-469.5189260655325,80.88622666722512,-84.82988295536593]
[177.0310856502517,663.9809715438986,261.50327924203305,-708.5431250876696,467.0380132620281,181.4509192089409,-37.15151425523848,635.0116960435249,882.0389729322873,-534.4893725108707]
[-1058.983438535667,390.9754848782779,1508.454706207631,363.79206833776055,275.1957888077916,-623.0196254444063,537.4147515744895,-218.67299199041472,-231.55887927232297,-99.98392390187095]
[-4685.773699057371,255.26635771944402,-153.10119543377468,-24.569787433435064,522.6588196148455,-375.9264880075217,-539.8743970690424,-470.0706533730587,-67.54765928695977,51.92673828087255]
[-2762.7905683587305,622.6539180572763,-405.00678943894866,-462.90978295573234,866.4534195252717,-919.4904224431655,-31.69129561997938,-782.0657727943528,516.2915128509082,237.11383873779545]


### PCA和SVD模型的关系
我们之前提到PCA和SVD有着密切的联系。事实上，可以使用SVD恢复出相同的主成分向量，并且应用相同的投影矩阵映射到主成分空间。  
在我们的例子中，SVD计算产生的右奇异向量等同于我们计算得到的主成分。可以通过在图像矩阵上计算SVD并比较右奇异向量和PCA的结果说明这一点。这里PCA和SVD的计算都可以通过分布式Row Matrix提供的函数完成：

In [24]:
val svd = matrix.computeSVD(10, computeU = true)
println( s"U dimension: (${svd.U.numRows}, ${svd.U.numCols})")
println( s"S dimension: (${svd.s.size}, )" )
println( s"V dimension: (${svd.V.numRows}, ${svd.V.numCols})" )

U dimension: (1054, 10)
S dimension: (10, )
V dimension: (2500, 10)


矩阵V和PCA的结果完全一样（不考虑正负符号和浮点数误差）。可以通过使用一个工具函数大致比较两个矩阵的向量数据来确定这一点：

In [27]:
def approxEqual( array1: Array[Double], array2: Array[Double], tolerance: Double = 1e-6 ): Boolean = {
    val bools = array1.zip( array2 ).map{ case(v1, v2) => if( math.abs(math.abs(v1) - math.abs(v2)) > 1e-6) false else true}
    bools.fold(true)(_&_)
}

In [28]:
println( approxEqual( Array(1.0, 2.0, 3.0), Array(3.0,2.0,1.0)))

false


In [29]:
println( approxEqual( Array(1.0, 2.0, 3.0), Array(1.0,2.0,3.0)))

true


In [31]:
println( approxEqual(svd.V.toArray, pc.toArray))

true


另外一个相关性体现在：矩阵U和向量S的乘积和PCA中用来把原始图像数据投影到10个主成分构成的空间中的投影矩阵相等：

In [32]:
val breezeS = breeze.linalg.DenseVector(svd.s.toArray)
val projectedSVD = svd.U.rows.map{ v =>
    val breezeV = breeze.linalg.DenseVector( v.toArray )
    val multV = breezeV :* breezeS
    Vectors.dense( multV.data )
}
projected.rows.zip( projectedSVD ).map{ 
    case( v1, v2 ) => approxEqual(v1.toArray, v2.toArray)
}.filter( b => true).count

1054

## 四、评价降维模型
####  在LFW数据集上估计SVD的K值
通过观察在我们的图像数据集上计算SVD得到的奇异值，可以确定奇异值每次运行结果相同，并且是按照递减的顺序返回，如下所示：

In [33]:
val sValues = (1 to 5).map{ i => matrix.computeSVD( i, computeU = false).s }
sValues.foreach(println)

[54091.009971103565]
[54091.009971103565,33757.70286798242]
[54091.009971103565,33757.702867982414,24541.193694775917]
[54091.00997110355,33757.70286798243,24541.19369477594,23309.584188883004]
[54091.00997110355,33757.702867982414,24541.19369477596,23309.58418888302,21803.09841158358]


为了估算SVD（和PCA）做聚类时的K值，以一个较大的k的变化范围绘制一个奇异值图时很有用的。可以看到没增加一个奇异值时增加的变化量是否基本保持不变

In [34]:
val svd300 = matrix.computeSVD( 300, computeU = false )
val sMatrix = new DenseMatrix( 1, 300, svd300.s.toArray )
csvwrite( new File("/tmp/s.csv"), sMatrix)